In [1]:

import pickle
import pandas as pd
import numpy as np


import torch as tc
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


In [2]:

with open('data/data_res.pickle', 'rb') as pk:
    data = pickle.load(pk)

In [20]:
from collections import defaultdict
data_reorg = defaultdict(list)

with open('data/preprocessing.pickle', 'rb') as pk:
    data_preprocessing = pickle.load(pk)

user_class = data_preprocessing['user_class']

for user in data['x']:
    for i,game in enumerate(data['x'][user]):
        if i+1 <= len(data['x'][user]) - 1:
            data_reorg['x_t'].append( data['x'][user][i] )
            data_reorg['y_t'].append( data['y'][user][i] )
            data_reorg['u_t'].append( data['u'][user][i] )
            data_reorg['x_t1'].append( data['x'][user][i+1] )
            data_reorg['y_t1'].append( data['y'][user][i+1] )
            data_reorg['class'].append( user_class[user][0] )
            
df = pd.DataFrame( data_reorg )


In [4]:

TRAINING_SAMPLE_SIZE = int(0.75*len(df))
train_idx = np.random.randint(0, len(df)-1, TRAINING_SAMPLE_SIZE)
verify_idx = np.setdiff1d(np.arange(len(df)), train_idx)

df.head()

,x_t,y_t,u_t,x_t1,y_t1
0,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.086603,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0.086603,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [5]:

class Net(tc.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = tc.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = tc.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

train_data = df.iloc[train_idx]
verify_data = df.iloc[verify_idx]
x_dim = 1
y_dim = len(train_data['y_t'].iloc[0])
u_dim = len(train_data['u_t'].iloc[0])

h_input_layer_dim = x_dim+u_dim
nn_hidden_width = 100
h_output_layer_dim = y_dim


In [6]:
x_set, u_set = np.asarray(train_data['x_t'].to_list()), np.asarray(train_data['u_t'].to_list())
y_set = np.asarray(train_data['y_t'].to_list())
x_set = np.reshape(x_set, (len(x_set), 1))
h_input_data = tc.tensor(np.hstack([x_set, u_set]), dtype=tc.float)
h_output_data = tc.tensor(y_set, dtype=tc.float)

In [7]:

net = nn.Sequential(nn.Linear(h_input_layer_dim, nn_hidden_width), 
                    nn.ReLU(), nn.Linear(nn_hidden_width, nn_hidden_width),
                    nn.ReLU(), nn.Linear(nn_hidden_width, nn_hidden_width),
                    nn.ReLU(), nn.Linear(nn_hidden_width, h_output_layer_dim))

optimizer = tc.optim.Adam(net.parameters(), lr=0.005)
loss_func = tc.nn.MSELoss()  # this is for regression mean squared loss


for _ in range(10):
    optimizer.zero_grad()                 # clear gradients for next train
    v_pred = net(h_input_data)            # input x and predict based on x  
    loss = loss_func(v_pred, h_output_data)   # must be (1. nn output, 2. target)
    
    loss.backward()                            # backpropagation, compute gradients
    optimizer.step()                      # apply gradients
    print(f'eps: {_} -- {loss}')

eps: 0 -- 0.004811255261301994
eps: 1 -- 0.0023802719078958035
eps: 2 -- 0.0012178040342405438
eps: 3 -- 0.0008796660695225
eps: 4 -- 0.000571892480365932
eps: 5 -- 0.0005510481423698366
eps: 6 -- 0.0005253558629192412
eps: 7 -- 0.0004938817583024502
eps: 8 -- 0.00047503120731562376
eps: 9 -- 0.0004516723856795579


In [8]:
x_set, u_set = np.asarray(verify_data['x_t'].to_list()), np.asarray(verify_data['u_t'].to_list())
y_set = np.asarray(verify_data['y_t'].to_list())
x_set = np.reshape(x_set, (len(x_set), 1))
h_input_verify = tc.tensor(np.hstack([x_set, u_set]), dtype=tc.float)
h_output_verify = tc.tensor(y_set, dtype=tc.float)

In [9]:
v_pred = net(h_input_verify)
loss = loss_func(v_pred, h_output_verify)
print(loss)

tensor(0.0004, grad_fn=<MseLossBackward>)


In [40]:
# f1
user_data = defaultdict(list)
cnt = 0
for i,row in df.iterrows():
    x = row.loc['x_t']
    y = row.loc['y_t']
    u = row.loc['u_t']
    x_next = row.loc['x_t1']

    
    if np.linalg.norm(y) != 0 and np.linalg.norm(u) != 0 and np.dot(y, u) != 0:
        ep = (x_next-x) / (np.dot(y, u)/(np.linalg.norm(y)*np.linalg.norm(u)))
    else:
        ep = 0

    user_data[row.loc['class']].append(ep)

#     if not (np.linalg.norm(y) != 0 and np.linalg.norm(u) != 0):
#         print(i, y, u)
#         cnt += 1
# print(cnt)

In [43]:
epsion_user = np.zeros(len(list(user_data.keys())))
for i,user in enumerate(user_data.keys()):
    epsion_user[i] = np.array(user_data[user]).mean()
    
print(epsion_user)

with open(f'data/epsilon.pickle', 'wb') as pk:
    pickle.dump(epsion_user, pk, protocol=pickle.HIGHEST_PROTOCOL) 

[ 1.64682487e+00  1.63971980e+00  9.05897257e-01  5.20887909e-01
  1.36371071e+00  9.75727489e-01  3.04661257e+00  8.90935654e-01
  3.71760520e+00  1.65594446e+00  1.55603544e+00  2.28185716e+00
  4.73746149e+00  8.17283382e-01  2.36801248e+00  1.27636493e+00
  2.29618325e+00  9.40485989e-01  2.67325025e-01  4.30048655e+00
  1.24149488e-01  1.66147287e+00  2.11286274e-01  2.36259984e+00
  9.07112334e-02  3.69853523e+00  3.66003637e+00  8.74430743e-01
  1.62936286e+00  1.55688002e+00  1.09509216e+00  6.64269753e-01
  1.11240794e+00  4.81468768e-01  6.01479646e-01  9.45826155e-01
  2.91523733e-01  3.82517031e+00  4.21584173e-01  2.32068232e-01
  5.67140168e-02  2.33726551e-01 -7.42390497e-05  5.49543190e-01
  1.40483824e+00  1.40089459e-01  1.18285918e-01  1.33986892e+00
  1.37335019e-02  2.80240542e-02]
